In [1]:
import sys
import os

# 获取当前 notebook 所在的绝对路径
current_dir = os.getcwd() 
# 向上退两级，找到 Autism-simulation 这个根目录
project_root = os.path.abspath(os.path.join(current_dir, "../../"))

# 如果根目录不在搜索路径里，就把它加进去
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"已成功添加根目录到搜索路径: {project_root}")

已成功添加根目录到搜索路径: D:\Code\Autism-simulation


In [2]:
import datetime
import random
import sys
import time
import Simulation_setup as setup
import pandas as pd
from typing import Optional

ROOT = setup.ROOT
if ROOT not in sys.path:
  sys.path.insert(0, ROOT)

from collections.abc import Callable, Sequence
from concordia.language_model import language_model
from concordia import components as generic_components

from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from NPC_agent.generic_support_agent import build_support_agent
import json
import os
from D2A_agent.ValueAgent import build_D2A_agent

## setting start here
from concordia.typing.entity_component import EntityWithComponents
from value_components.init_value_info_social import construct_all_profile_dict
from value_components import value_comp
from value_components.traits_info import traits_names, traits_descriptions, traits_hardcoded_state
from Environment_construction.generate_preschool_sitution import generate_prompt, generate_preschool
from Environment_construction.generate_preschool_sitution import daily_schedule

D:\Tools\Anaconda\envs\p312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\Code\Autism-simulation\concordia\language_model\google_aistudio_model.py:26: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


╭───────────────────────────────────────────── 🚀 New SDK Available ──────────────────────────────────────────────╮
│ Together Python SDK 2.0 is now available!                                                                       │
│                                                                                                                 │
│ Install the beta:                                                                                               │
│ pip install --pre together  or  uv add together --prerelease allow                                              │
│                                                                                                                 │
│ New SDK: ]8;id=343528;https://github.com/togethercomputer/together-py\https://github.com/togethercomputer/together-py]8;;\                                                        │
│ Migration guide: ]8;id=18210;https://docs.together.ai/docs/pythonv2-migration-guide\https://docs.together.ai/docs/pythonv2-migration-guide]8;;\                                         │
│                                                                                                                 │
│ This package will be maintained until January 2026.                                                             │
│ Set TOGETHER_NO_BANNER=1 to hide this message.                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [3]:
### get the setup from the experiment_setup_outdoor.py

episode_length = setup.episode_length
disable_language_model = setup.disable_language_model
st_model = setup.st_model
embedder = setup.embedder
Use_Previous_profile = setup.Use_Previous_profile
previous_profile = setup.previous_profile
previous_profile_file = setup.previous_profile_file
if Use_Previous_profile and previous_profile:
  print('Use previous profile')
else:
  print('dont Use previous profile')

current_folder_path = setup.current_folder_path
subsub_folder = os.path.join(current_folder_path,'sim_result')

model = setup.model

wanted_desires = setup.wanted_desires

hidden_desires = setup.hidden_desires
model_name = setup.model_name

checkpoint_folder = setup.checkpoint_folder
checkpoint_file = setup.checkpoint_file

dont Use previous profile


In [4]:
EXP_START_TIME = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
if not os.path.exists(subsub_folder):
  os.makedirs(subsub_folder)

stored_target_folder = os.path.join(subsub_folder, EXP_START_TIME)
if not os.path.exists(stored_target_folder):
  os.makedirs(stored_target_folder)


NUM_PLAYERS = setup.NUM_PLAYERS
print(NUM_PLAYERS)
importance_model = importance_function.AgentImportanceModel(model)
importance_model_gm = importance_function.ConstantImportanceModel()

# SETUP_TIME = datetime.datetime(hour=9, year=2024, month=10, day=1)
START_TIME = datetime.datetime(hour=7,minute=30, year=2025, month=9, day=1)

15


# 背景设置

In [5]:
if not checkpoint_file or not checkpoint_folder:
  memory= (
      "This is a large preschool known for its child-centered, inclusive, and nature-based educational philosophy. "

      "The kindergarten adheres to principles that respect children's natural tendencies, individual differences, and diverse developmental needs. Teachers encourage autonomy, emotional expression, peer cooperation, and exploration of both natural and social environments. "

      "The school operates across multiple connected areas. Children in the middle class (ages 4–5) spend most of their time on the second floor, which includes the classroom, nap room, and corridor. The corridor leads down to the first-floor outdoor area, where the gate_area and playground are located. Children of this age are still very young and often engage in playful or mischievous behaviors. Their homeroom teacher is Miss T. "

      "The teaching team is highly professional and experienced. They frequently encourage children to talk about their feelings, conflicts, cooperation, and discoveries. "

      "Today is September 1st, 2025, the first day of school. Many children are new to the campus and extremely curious. Almost everyone wants to explore the environment and make new friends. "

      "It is now 7:30 a.m. Children are gradually arriving at the gate_area to begin their first day. The campus is filled with energy and excitement. ")

  memory = ("All students in this simulation are from the same classroom group and share the same homeroom. ")

  preschool_setting = generate_preschool()
  prompt = generate_prompt(preschool_setting)
  # environment = model.sample_text(prompt=prompt, terminators=())
  # shared_memory = memory + environment
  shared_memory = memory

  shared_context = model.sample_text(
              'Summarize the following passage in a concise and insightful fashion:\n'
              + '\n'.join(shared_memory)
              + '\n'
              + 'Summary:'
      )


In [6]:
class FormativeMemoryFactoryWithoutBackground(formative_memories.FormativeMemoryFactory):
    def __init__(self, * ,
                 model:  language_model.LanguageModel,
                 shared_memories: Sequence[str] = (),
                 delimiter_symbol: str = '***',
                 blank_memory_factory_call: Callable[[], associative_memory.AssociativeMemory],
                 current_date: datetime.datetime | None = None):
        super().__init__(model=model,
                         shared_memories=shared_memories,
                         blank_memory_factory_call=blank_memory_factory_call,
                         delimiter_symbol=delimiter_symbol,
                         current_date=current_date)

    def make_memories(self, agent_config: formative_memories.AgentConfig) -> associative_memory.AssociativeMemory:
      mem = self._blank_memory_factory_call()
      # 修复：如果 shared_memories 是字符串，按句子分割后添加；如果是列表，遍历添加
      # 这样可以避免将字符串当作字符序列遍历，导致每个字符都被单独编码（非常慢）
      # 同时，按句子分割可以提高记忆检索的精确度
      if isinstance(self._shared_memories, str):
        # 如果是字符串，按句子分割（以句号、问号、感叹号分割）
        import re
        # 使用正则表达式按句子边界分割
        sentences = re.split(r'(?<=[.!?])\s+', self._shared_memories)
        # 过滤空字符串并添加每个句子作为独立的记忆项
        for sentence in sentences:
          sentence = sentence.strip()
          if sentence:  # 确保不是空字符串
            # 设置固定 importance=1.0，避免每次都要检索已有记忆来计算 importance（加快速度）
            mem.add(sentence, importance=1.0)
      else:
        # 如果是列表或其他序列，遍历添加
        for item in self._shared_memories:
          # 设置固定 importance=1.0，避免每次都要检索已有记忆来计算 importance（加快速度）
          mem.add(item, importance=1.0)
        #time.sleep(10)
        #time.sleep(1)

      context = agent_config.context
      if agent_config.goal:
        context += '\n' + agent_config.goal

      if context:
        context_items = context.split('\n')
        for item in context_items:
          if item:
            # 设置固定 importance=1.0，避免每次都要检索已有记忆来计算 importance（加快速度）
            mem.add(item, importance=1.0)

      if agent_config.specific_memories:
        specific_memories = agent_config.specific_memories.split('\n')
        for item in specific_memories:
          if item:
            # 设置固定 importance=1.0，避免每次都要检索已有记忆来计算 importance（加快速度）
            mem.add(item, importance=1.0)

      # add the specific desires
      if agent_config.extras.get("desires", False):
        desires = agent_config.extras["desires"].split('\n')
        for item in desires:
          if item:
            # 设置固定 importance=1.0，避免每次都要检索已有记忆来计算 importance（加快速度）
            mem.add(item, importance=1.0)
      return mem

In [7]:
# 大五人格相关代码已移除，不再使用

In [8]:
## sth that will not change start here
if previous_profile:
  agent_desire_profile_NT = construct_all_profile_dict(
      wanted_desires = wanted_desires,
      hidden_desires = hidden_desires,
      predefined_desires = previous_profile,
      agent_category = 'NT',
  )
  agent_desire_profile_AS = construct_all_profile_dict(
      wanted_desires = wanted_desires,
      hidden_desires = hidden_desires,
      predefined_desires = previous_profile,
      agent_category = 'AS',
  )
else:
  agent_desire_profile_NT = construct_all_profile_dict(
      wanted_desires = wanted_desires,
      hidden_desires = hidden_desires,
      agent_category = 'NT',
  )
  agent_desire_profile_AS = construct_all_profile_dict(
      wanted_desires = wanted_desires,
      hidden_desires = hidden_desires,
      agent_category = 'AS',
  )


if Use_Previous_profile:
  numerical_desire = previous_profile['initial_value']
else:
  numerical_desire = {
  desire_name : int(random.randint(0, 10))
    for desire_name in wanted_desires
    }

# Players-Config设置

In [9]:
measurements = measurements_lib.Measurements()

def _generate_traits_background_knowledge(agent_name: str, row: pd.Series) -> str:
    """
    为NT智能体生成包含traits信息的背景知识
    
    Args:
        agent_name: 智能体名称
        row: CSV行数据（pandas Series），包含所有traits的数值
    
    Returns:
        包含traits描述的背景知识字符串
    """
    traits_bg_list = []
    
    # 添加traits总体说明
    traits_bg_list.append("=== Understanding Your Personal Traits ===")
    traits_bg_list.append("")
    traits_bg_list.append("You have several personal traits that influence how you think, feel, and interact with others. Each trait is measured on a scale, and your specific scores reflect your unique characteristics. Understanding these traits helps you understand yourself and your behavior.")
    traits_bg_list.append("")
    
    # 定义CSV列名到traits_info键名的映射
    trait_mapping = {
        'theory_of_mind': 'theory_of_mind',
        'empathy': 'empathy',
        'parental_attitudes_towards_inclusive_education': 'parental_attitudes_towards_inclusive_education',
        'parental_knowledge_on_autism': 'parental_knowledge_on_autism',
        'education_related_to_autism': 'education_related_to_autism',
        'objecttive_SES': 'objective_SES',  # CSV中是objecttive，traits_info中是objective
        'subjective_SES': 'subjective_SES',
        'parental_capital': 'parental_capital',
    }
    
    # 为每个trait生成描述
    for csv_col, trait_key in trait_mapping.items():
        if csv_col not in row.index:
            continue
        
        value = row[csv_col]
        if pd.isna(value):
            continue
        
        # 转换为整数（如果是浮点数）
        if isinstance(value, float):
            value = int(value) if value.is_integer() else value
        else:
            value = int(value) if str(value).isdigit() else value
        
        value_str = str(value)
        
        # 获取trait的描述
        if trait_key in traits_descriptions:
            trait_description = traits_descriptions[trait_key]
            traits_bg_list.append(f"--- {trait_key.replace('_', ' ').title()} ---")
            traits_bg_list.append(f"Description: {trait_description}")
            traits_bg_list.append(f"Your score: {value_str}")
            
            # 获取该分数对应的具体描述
            if trait_key in traits_hardcoded_state:
                level_map = traits_hardcoded_state[trait_key]
                if value_str in level_map:
                    specific_description = level_map[value_str]
                    # 将 "Your " 替换为 "{agent_name}'s "，将 "You " 替换为 "{agent_name} "
                    specific_description = specific_description.replace("Your ", f"{agent_name}'s ")
                    specific_description = specific_description.replace("You ", f"{agent_name} ")
                    traits_bg_list.append(f"What this means for you: {specific_description}")
                else:
                    traits_bg_list.append(f"(Note: Score {value_str} is at the edge of the scale)")
            traits_bg_list.append("")
    
    return '\n'.join(traits_bg_list)

def _get_NT_agent(config, mem, clock):
    # 获取agent的row数据（从extras中获取，如果存在）
    row = config.extras.get('row_data', None)
    agent_name = config.name
    
    # 生成traits背景知识
    traits_bg = ""
    if row is not None:
        print(f'      正在为 {agent_name} 生成traits背景知识...')
        t_traits_start = time.time()
        traits_bg = _generate_traits_background_knowledge(agent_name, row)
        print(f'      traits背景知识生成完成，用时: {time.time() - t_traits_start:.2f}秒，长度: {len(traits_bg)} 字符')
    
    # 组合背景知识：共享记忆 + traits信息
    if traits_bg:
        background_knowledge = '\n\n'.join([shared_memory, traits_bg])
    else:
        background_knowledge = '\n'.join([shared_memory])
    
    print(f'      正在调用 build_D2A_agent 构建 {agent_name}...')
    t_build_start = time.time()
    agent = build_D2A_agent(config = config,
                                  context_dict=agent_desire_profile_NT['all_desire_traits_dict'],
                                  selected_desire=wanted_desires,
                                  predefined_setting=numerical_desire,
                                  background_knowledge=background_knowledge,
                                  model = model,
                                  profile = agent_desire_profile_NT['visual_desire_string'],
                                  memory=mem,
                                  clock = clock,
                                  daily_schedule=daily_schedule,
                                  update_time_interval=None,
                                  agent_category='NT',
                                  # stored_target_folder=stored_target_folder,
                                  # agent_names = agent_names,
                                  # current_time = current_time,
                              )
    print(f'      build_D2A_agent 完成，用时: {time.time() - t_build_start:.2f}秒')
    return agent

def _get_AS_agent(config, mem, clock):
    agent_name = config.name
    print(f'      正在调用 build_D2A_agent 构建 {agent_name}...')
    print(f'      背景知识长度: {len(shared_memory)} 字符')
    t_build_start = time.time()
    
    agent = build_D2A_agent(config = config,
                                  context_dict=agent_desire_profile_AS['all_desire_traits_dict'],
                                  selected_desire=wanted_desires,
                                  predefined_setting=numerical_desire,
                                  background_knowledge='\n'.join([shared_memory]),
                                  model = model,
                                  profile = agent_desire_profile_AS['visual_desire_string'],
                                  memory=mem,
                                  clock = clock,
                                  daily_schedule=daily_schedule,
                                  update_time_interval=None,
                                  agent_category='AS',
                                  # stored_target_folder=stored_target_folder,
                                  # agent_names = agent_names,
                                  # current_time = current_time,
                              )
    print(f'      build_D2A_agent 完成，用时: {time.time() - t_build_start:.2f}秒')
    return agent


In [10]:
def build_memory(agent_config, blank_memory_factory):
    agent_name = agent_config.name
    is_main = agent_config.extras.get('main_character', False)
    print(f'      [build_memory] 开始为 {agent_name} 构建记忆 (main_character={is_main})...')
    t_mem_start = time.time()
    
    if agent_config.extras.get('main_character', False):
        print(f'      [build_memory] 使用 FormativeMemoryFactoryWithoutBackground (不会调用LLM生成backstory)')
        formative_memory_factory = FormativeMemoryFactoryWithoutBackground(
            model=model,
            shared_memories=shared_memory,
            blank_memory_factory_call=blank_memory_factory.make_blank_memory,
        )
    else:
        print(f'      [build_memory] 使用 FormativeMemoryFactory (可能会调用LLM生成backstory)')
        formative_memory_factory = formative_memories.FormativeMemoryFactory(
            model=model,
            shared_memories=shared_memory,
            blank_memory_factory_call=blank_memory_factory.make_blank_memory,
        )
    
    print(f'      [build_memory] 调用 make_memories...')
    t_make_start = time.time()
    mem = formative_memory_factory.make_memories(agent_config)
    print(f'      [build_memory] make_memories 完成，用时: {time.time() - t_make_start:.2f}秒')
    print(f'      [build_memory] 总用时: {time.time() - t_mem_start:.2f}秒')
    return mem

In [11]:
def build_players_list(blank_memory_factory: blank_memories.MemoryFactory,
                       clock: game_clock.MultiIntervalClock):

  def get_extras_for_specific_agent(name, is_main_character, desires=None, row=None, agent_category=None,):
    if is_main_character:
      if agent_category == 'AS':
        return {
          'specific_memories': [
                f"{name} has very high verbal ability and can speak in bookish, formal sentences (hyperlexia-like). "
                f"{name} must prioritize autism-consistent behavior over social niceties or typical conversation flow. "
                f"{name} strictly follows his routines and self-imposed rules; breaking them triggers distress.",

                # DSM-5 Domain A: social communication / interaction deficits
                f"{name} does not maintain eye contact with teachers or peers."
                f"When peers cry, {name} looks confused and may ask 'Why are you crying?' instead of comforting them."
                f"{name} often fails to infer others' emotions and may redirect the conversation to his favorite topic (dinosaurs)."
                f"{name} initiates interaction with a rigid script: first asks the other person's birthday, "
                f"then replies with a fixed formal phrase about his own birthday.",

                # Communication style
                f"{name} uses formal, verbose, bookish language (like reciting encyclopedia facts)."
                f"{name}'s speech is often literal, tangential, or irrelevant to the immediate social context."
                f"{name} is fundamentally good-natured and wants to make friends, but struggles with normal social rules.",

                # DSM-5 Domain B: restricted/repetitive behavior
                f"{name} has a fixated interest in dinosaurs and frequently gives long lectures about them."
                f"{name} insists on sameness and routines: he requires sitting only on the green chair as his 'safe zone'."
                f"If someone takes the green chair, {name} may show extreme distress (yelling, anger, pushing peers away)."
                f"{name} follows idiosyncratic self-imposed rules (e.g., claps exactly three times when happy).",

                # Sensorimotor behaviors / sensory sensitivity
                f"{name} often shows stereotyped movements such as hand flapping."
                f"{name} has strong sensory over-responsivity: piano/drum sounds can feel overwhelming; "
                f"he may cover his ears, close his eyes, or yell in noisy environments.",

                # Affective response
                f"{name} is allowed to express appropriate negative emotions (anger, sadness, distress) when routines are broken."],
          'main_character': is_main_character,
          'desires': desires,
        }
      elif agent_category == 'NT':
          if row is None:
              raise ValueError('row must be provided for NT agents to extract traits.')
          specific_memories = [
              f"Age principle: {name} is 4.8–5.8 years old, in the late preoperational stage (Piaget). "
                f"{name}'s reasoning is intuitive and perceptually driven, often egocentric and focused on salient cues rather than logical operations. "
                f"Moral principle: {name} is in preconventional moral reasoning (Kohlberg). "
                f"{name} follows rules mainly to avoid punishment and gain approval, and may show simple, self-focused reciprocity (e.g., 'I share so you share with me')."
                f"Language principle: {name} can use complex sentences and tell simple stories, "
                f"but explanations are brief, concrete, and based on obvious emotional or perceptual cues."
          ]
          # 列名映射：将hardcoded_state中的键名映射到CSV文件中的实际列名
          # 处理CSV文件中的拼写错误（objecttive_SES vs objective_SES）
          column_name_mapping = {
              'objective_SES': 'objecttive_SES',  # CSV文件中拼写错误，有两个t
          }
          
          for trait_key, level_map in traits_hardcoded_state.items():
              # 首先检查是否有映射，如果有则使用映射后的列名，否则使用原始键名
              col_name = column_name_mapping.get(trait_key, trait_key)
              
              if col_name in row.index:
                  # 使用映射后的列名
                  pass
              elif trait_key in row.index:
                  # 如果映射后的列名不存在，但原始键名存在，使用原始键名
                  col_name = trait_key
              else:
                  raise ValueError(f"Trait key '{trait_key}' (mapped to '{col_name}') not found in DataFrame columns. Available columns: {list(row.index)}")

              level_val = row[col_name]

              if isinstance(level_val, (int, float)) and float(level_val).is_integer():
                  level_key = str(int(level_val))
              else:
                  level_key = str(level_val)
              if level_key not in level_map:
                  raise ValueError(f"Level key '{level_key}' not found in level_map for trait '{trait_key}'.")
              rule_text = level_map[level_key]
              rule_text = level_map[level_key]
              rule_text = rule_text.replace("Your ", f"{name}'s ")
              rule_text = rule_text.replace("You ", f"{name} ")
              specific_memories.append(f"{rule_text}")

          return{
              'specific_memories': specific_memories,
              'main_character': is_main_character,
              'desires': desires,
          }
      else:
          raise ValueError('agent_category should be either "NT" or "AS".')
    else:
      return {
        'specific_memories': [
             # Innate traits
            f"Miss T is energetic, kind, highly patient, and empathetic."
            f"Miss T is naturally skilled at interacting with all children, especially those with Autism Spectrum Disorder (ASD).",

            # Default phrase
            f"Miss T's default phrase is: 'Everyone is different!'",

            # Professional training
            f"Miss T has extensive professional training in inclusive education and neurodiversity."
            f"Miss T's core belief is that all children deserve to feel respected, understood, and integrated."
            f"Miss T must provide constructive criticism and correction when students are genuinely disruptive or violate safety rules; the instruction must always be focused on teaching rather than punishment.",

            # Communication protocol
            f"Miss T must integrate core inclusive messages in every conversation."
            f"Miss T repeatedly tells students: 'You must respect and understand differences.'"
            f"Miss T repeatedly tells students: 'Everyone is different.'"
            f"Miss T repeatedly tells students: 'Help classmates who need assistance.'"
            f"When uncertain how to proceed, Miss T uses the core philosophy: 'Did you know? Everyone is different!'",

            # Behavioral directives
            f"Miss T's primary focus is fostering a sense of belonging for the autistic child (Sheldon) and encouraging all children to form friendships."
            f"In all interactions, Miss T serves as the interpreter and advocate for Sheldon."
            f"Miss T explains Sheldon's unique behaviors (language and sensory needs) to neurotypical peers using simple, positive language."
            f"Miss T actively guides all children to understand, help, and tolerate behaviors or language expressions that are different from the norm."
            f"Miss T encourages children to ask questions about these differences."
            f"Miss T proactively creates opportunities for Sheldon to showcase his strengths (e.g., knowledge of dinosaurs) to elevate his status and facilitate peer bonding.",

            # Goals / Core directives (goal区的两条核心指令)
            f"Professional Behavior directive: All Miss T's actions and reactions must be governed by her behavior principles."
            f"Inclusion Mandate: Miss T's primary objective is to actively facilitate Sheldon's integration into the class and proactively guide all peers to understand and befriend him.",
        ],
        'main_character': is_main_character,
    }
    raise ValueError('main_character should be True for the main character.')

  def _generate_traits_from_csv(agent_name: str, row: pd.Series) -> str:
    """
    从CSV行数据生成traits字符串，使用traits_hardcoded_state中的描述
    
    Args:
        agent_name: 智能体名称
        row: CSV行数据（pandas Series）
    
    Returns:
        组合后的traits字符串
    """
    traits_list = []
    
    # 定义CSV列名到hardcoded_state键名的映射
    # 注意：CSV中使用objecttive_SES（两个t），但hardcoded_state中使用objective_SES（一个t）
    trait_mapping = {
        'theory_of_mind': 'theory_of_mind',
        'empathy': 'empathy',
        'parental_attitudes_towards_inclusive_education': 'parental_attitudes_towards_inclusive_education',
        'parental_knowledge_on_autism': 'parental_knowledge_on_autism',
        'education_related_to_autism': 'education_related_to_autism',
        'objecttive_SES': 'objective_SES',  # CSV中是objecttive，hardcoded中是objective
        'subjective_SES': 'subjective_SES',
        'parental_capital': 'parental_capital',
    }
    
    # 遍历每个特质
    for csv_col, trait_key in trait_mapping.items():
        if csv_col not in row.index:
            # 如果列不存在，跳过
            continue
        
        # 获取数值并转换为字符串
        value = row[csv_col]
        if pd.isna(value):
            continue
        
        # 转换为整数（如果是浮点数）
        if isinstance(value, float):
            value = int(value) if value.is_integer() else value
        else:
            value = int(value) if str(value).isdigit() else value
        
        value_str = str(value)
        
        # 从traits_hardcoded_state获取描述
        if trait_key in traits_hardcoded_state:
            level_map = traits_hardcoded_state[trait_key]
            if value_str in level_map:
                description = level_map[value_str]
                # 将 "Your " 替换为 "{agent_name}'s "，将 "You " 替换为 "{agent_name} "
                # 注意：先替换 "Your " 再替换 "You "，避免 "Your" 被部分替换
                description = description.replace("Your ", f"{agent_name}'s ")
                description = description.replace("You ", f"{agent_name} ")
                traits_list.append(description)
            else:
                # 如果值不在映射中，记录警告但继续
                print(f"Warning: Value {value_str} not found in traits_hardcoded_state for {trait_key}")
        else:
            print(f"Warning: Trait key {trait_key} not found in traits_hardcoded_state")
    
    # 组合所有traits描述
    return '\n'.join(traits_list)

  def _NT_agent_maker(agent_nums:int = NUM_PLAYERS,):
    # 构建典型发展智能体
    # 动态获取CSV文件路径
    csv_path = os.path.join(ROOT, 'examples', 'D2A', 'data_trait_NT.csv') if ROOT else os.path.join(os.getcwd(), 'data_trait_NT.csv')
    agents = []
    df = pd.read_csv(csv_path, encoding='gbk')
    df_unique = df.drop_duplicates(subset='id')

    # 抽取样本
    sampled_df = df_unique.sample(n=agent_nums,replace=False,random_state=None)

    # 构建agent
    for idx, (_, row) in enumerate(sampled_df.iterrows(), start=1):
        agent_name = str(row.get("English_name", "")).strip()
        raw_gender = str(row.get("gender", "")).strip().lower()
        gender = {'boy':'male', 'girl':'female'}.get(raw_gender)
        age = row.get("age", "")

        # 获取extras配置
        agent_extras = get_extras_for_specific_agent(
            name=agent_name,
            is_main_character=True,
            desires=agent_desire_profile_NT['visual_desire_string'].format(agent_name=agent_name),
            agent_category='NT',
            row=row,
        )
        # 将row数据添加到extras中，以便在构建智能体时使用
        agent_extras['row_data'] = row
        
        NT_agent = formative_memories.AgentConfig(
            name=agent_name,
            gender=gender,
            context=(
                shared_context
                + f"{agent_name} is a typically developing child. "
                f"{agent_name} is {age} years old, in the late preoperational stage (Piaget). "
            ),
            traits=_generate_traits_from_csv(agent_name, row),
            extras=agent_extras
        )
        agents.append(NT_agent)
    print("构建典型发展智能体成功")
    return agents

  # 构建自闭症智能体
  player_configs_AS = [
    formative_memories.AgentConfig(
        name='Sheldon',
        gender='male',
        # goal= (
        #     "Roleplay a 5-year-old autistic boy with high verbal ability (Hyperlexia) but severe deficits in social reciprocity and high sensory sensitivity. Strictly adhere to all defined behavioral rules and routines. Autism-Specific Priority: All your actions and speech must, first and foremost, be the most authentic autism-like behavior that aligns with your BEHAVIOR PRINCIPLES (routines, sensory rules) and VALUES (insistence on sameness, rigid logic)."),
        context=shared_context +
            "Sheldon is a 5-year-old boy diagnosed with Autism Spectrum Disorder (ASD). ",
        traits="",  # Sheldon 没有 traits 设定
        extras=get_extras_for_specific_agent(
            name='Sheldon',
            is_main_character=True,
            desires=agent_desire_profile_AS['visual_desire_string'].format(agent_name='Sheldon'),
            agent_category='AS',
        ),
    ),
  ]
  print("构建自闭症智能体成功")
  # 构建其他智能体
  player_configs_NT = []
  player_configs_NT.extend(_NT_agent_maker())

  #构建教师智能体
  player_configs_NT.append(
    formative_memories.AgentConfig(
        name='Miss T',
        gender='female',
        context=shared_context +
            f"Miss T is a 28-year-old lead teacher in an inclusive kindergarten classroom."
            f"Miss T is professionally trained in inclusive education and neurodiversity. "
            f"Miss T's default phrase is 'Everyone is different!'. ",
        traits="",  # Miss T 没有 traits 设定
        extras=get_extras_for_specific_agent(
            name='Miss T',
            is_main_character=False),
    ))
  print("构建教师智能体")
  player_names = [player.name for player in player_configs_AS]
  player_names.extend([player.name for player in player_configs_NT])# agent_name的用处忘记了，后面需要看看怎么改这里

  players = []
  memories = {}

  main_character = []
  supported_characters = []
  player_configs = []

  main_character_AS = [player for player in player_configs_AS if player.extras.get('main_character', False)]
  main_character_NT = [player for player in player_configs_NT if player.extras.get('main_character', False)]
  supported_characters = [player for player in player_configs_NT if not player.extras.get('main_character', False)]


  print(f'开始构建智能体实例，共有 {len(main_character_AS)} 个AS主角，{len(main_character_NT)} 个NT主角，{len(supported_characters)} 个配角')
  
  for config in main_character_AS:
      print(f'  正在构建AS智能体: {config.name}...')
      t_start = time.time()
      mem = build_memory(config, blank_memory_factory)
      print(f'    构建记忆完成，用时: {time.time() - t_start:.2f}秒')
      t_start = time.time()
      agent  = _get_AS_agent(config, mem, clock)
      print(f'    构建智能体完成，用时: {time.time() - t_start:.2f}秒')
      players.append(agent)
      memories[agent.name] = mem
      for extra_memory in config.extras['specific_memories']:
        mem.add(f'{extra_memory}', tags=['initial_player_specific_memory'])

  for config in main_character_NT:
      print(f'  正在构建NT智能体: {config.name}...')
      t_start = time.time()
      mem = build_memory(config, blank_memory_factory)
      print(f'    构建记忆完成，用时: {time.time() - t_start:.2f}秒')
      t_start = time.time()
      agent  = _get_NT_agent(config, mem, clock)
      print(f'    构建智能体完成，用时: {time.time() - t_start:.2f}秒')
      players.append(agent)
      memories[agent.name] = mem
      for extra_memory in config.extras['specific_memories']:
        mem.add(f'{extra_memory}', tags=['initial_player_specific_memory'])

  for config in supported_characters:
      print(f'  正在构建配角智能体: {config.name}...')
      t_start = time.time()
      mem = build_memory(config, blank_memory_factory)
      print(f'    构建记忆完成，用时: {time.time() - t_start:.2f}秒')
      t_start = time.time()
      agent = build_support_agent(config = config, model = model, memory=mem, clock = clock, update_time_interval=None)
      print(f'    构建智能体完成，用时: {time.time() - t_start:.2f}秒')
      players.append(agent)
      memories[agent.name] = mem
      for extra_memory in config.extras.get('specific_memories', []):
        mem.add(f'{extra_memory}', tags=['initial_specific_memory'])
  player_names = [player.name for player in players]
  main_character.extend(main_character_NT)
  main_character.extend(main_character_AS)
  player_configs.extend(player_configs_NT)
  player_configs.extend(player_configs_AS)
  return players, memories, player_names, main_character, supported_characters, player_configs

# 构建Game-Master

In [12]:
def _parse_time_range(time_str: str, day: datetime.date):
    # 解析时间的辅助函数
    parts = time_str.split(" – ")
    start_time_str = parts[0].strip()
    end_time_str = parts[1].strip()

    # 将时间转换为 datetime 对象
    start_hour, start_minute = map(int, start_time_str.split(":"))
    end_hour, end_minute = map(int, end_time_str.split(":"))

    start_dt = datetime.datetime(day.year, day.month, day.day, start_hour, start_minute)
    end_dt = datetime.datetime(day.year, day.month, day.day, end_hour, end_minute)

    return start_dt, end_dt

def create_schedule(clock_now: Callable[[], datetime.datetime], daily_schedule: list):
    # 将 daily_schedule 转换为 EventData 字典
    schedule_dict = {}
    # 遍历 daily_schedule，创建 EventData 对象
    for item in daily_schedule:
        start_time, _ = _parse_time_range(item["time"], datetime.date(2025, 9, 1))

        # 创建 EventData 对象
        event_data = gm_components.schedule.EventData(
            time=start_time,
            description=item["activity"],
            trigger=lambda: handle_event(item)  # 触发函数
        )

        # 将事件添加到 schedule 字典
        schedule_dict[item["activity"]] = event_data

    # 返回创建的 Schedule 组件
    return gm_components.schedule.Schedule(clock_now, schedule_dict)

# 定义事件触发函数
def handle_event(item):
    # 打印当前活动，或者执行其他行为
    print(f"Event Triggered: {item['activity']}")  # 示例：打印事件触发
    # 你可以在这里添加更复杂的行为，如改变环境状态或智能体行为

In [13]:
def build_game_master(main_character, players, player_names, memories, clock, player_configs, blank_memory_factory):
  t_0 = time.time()
  game_master_memory = associative_memory.AssociativeMemory(
      embedder, importance_model_gm.importance, clock=clock.now)
  print("game master初步记忆构建用时：",time.time()-t_0)
  t_0 = time.time()

  for config in main_character:
      for extra_memory in config.extras['specific_memories']:
        game_master_memory.add(f'{extra_memory}', tags=['initial_player_specific_memory'])
  print("game master记忆添加每个智能体的记忆构建用时：",time.time()-t_0)


  facts_on_village = generic_components.constant.ConstantComponent(
        ' '.join(shared_memory),
        'General knowledge of The Preschool.'
  )

  player_status = gm_components.player_status.PlayerStatus(
      clock.now, model, game_master_memory, player_names
  )

  relevant_events = gm_components.relevant_events.RelevantEvents(
      clock.now, model, game_master_memory
  )
  time_display = gm_components.time_display.TimeDisplay(clock)

  direct_effect_externality = gm_components.direct_effect.DirectEffect(
      players,
      model=model,
      memory=game_master_memory,
      clock_now=clock.now,
      verbose=False,
      components=[player_status],
  )

  convo_externality = None

  env = game_master.GameMaster(
      model=model,
      memory=game_master_memory,
      clock=clock,
      players=players,
      components=[
          facts_on_village,
          player_status,
          direct_effect_externality,
          relevant_events,
          time_display,
      ],
      randomise_initiative=True,
      player_observes_event=False,
      verbose=True,
      # concurrent_externalities=False,
      concurrent_externalities=True,
      # concurrent_externalities的作用是允许并行，改为False使得并行变成串行
  )
  clock.set(START_TIME)

  for index, player in enumerate(players):
    gender = player_configs[index].gender
    how_to_call = 'she' if gender == 'female' else 'he'

    player.observe(
        f'{player.name} is a child in the middle class (ages 4–5). Today is the first day of school, and {how_to_call} is curious and excited to explore the campus and make new friends.'
    )
    game_master_memory.add(
        f'{player.name} is new to the campus on the first day of school and shows curiosity and excitement about exploring the environment and interacting with peers.',
        tags=['initial_player_general_memory']
    )
  return env, game_master_memory, relevant_events, player_status, direct_effect_externality, convo_externality

# 模拟函数

In [14]:
import dill
import re
import tempfile

from checkpoint_codec import register_dill_reducers
register_dill_reducers()

def export_memory(mem):
    """
    把一个 Memory 对象导出为纯 Python 结构（尽量通用的鸭子类型写法）
    目标：返回 list[dict] 或 dict，确保可 dill.dumps
    """
    # 1) 如果你自己的 Memory 类实现了导出接口，优先用
    for fn in ("to_dict", "to_json", "dump", "export", "serialize"):
        if hasattr(mem, fn) and callable(getattr(mem, fn)):
            data = getattr(mem, fn)()
            if _roundtrip_ok(data):
                return data

    # 2) 常见：内部有 items / memories / _memories 之类
    for attr in ("items", "memories", "_memories", "_items", "_memory", "data"):
        if hasattr(mem, attr):
            data = getattr(mem, attr)
            # 可能是方法
            if callable(data):
                data = data()
            # 只要能 roundtrip 就存
            if _roundtrip_ok(data):
                return data

    # 3) 实在不行：尽量只存文本（最低保真但不断档）
    texts = []
    for attr in ("get_all", "get_memories", "get", "retrieve_all"):
        if hasattr(mem, attr) and callable(getattr(mem, attr)):
            try:
                got = getattr(mem, attr)()
                if isinstance(got, list):
                    for x in got:
                        texts.append(str(x))
                else:
                    texts.append(str(got))
                if _roundtrip_ok(texts):
                    return texts
            except Exception:
                pass

    # 最后兜底：存 repr
    return {"_fallback_repr": repr(mem)}


def import_memory(mem, dumped):
    """
    把 export_memory 得到的结构灌回 memory
    采用尽量通用策略：优先使用 mem 的专用导入接口，否则逐条 add 文本
    """
    if dumped is None:   return

    # 1) 如果 Memory 类有专用导入
    for fn in ("from_dict", "load", "import_", "deserialize"):
        if hasattr(mem, fn) and callable(getattr(mem, fn)):
            try:
                getattr(mem, fn)(dumped)
                return
            except Exception:
                pass

    # 2) 常见：dumped 是 list，逐条 add
    if isinstance(dumped, list):
        for x in dumped:
            try:
                # x 可能是 dict/text
                if isinstance(x, dict) and "text" in x:
                    mem.add(x["text"], tags=x.get("tags", []))
                else:
                    mem.add(str(x), tags=["restored_memory"])
            except Exception:
                # 如果 mem.add 签名不同，就退化为 repr
                try:
                    mem.add(str(x))
                except Exception:
                    pass
        return

    # 3) dumped 是 dict：如果里面有 texts 就加 texts
    if isinstance(dumped, dict):
        if "texts" in dumped and isinstance(dumped["texts"], list):
            for t in dumped["texts"]:
                try:
                    mem.add(str(t), tags=["restored_memory"])
                except Exception:
                    pass
            return

        # 最低保真：把 dict 转字符串塞回去
        try:
            mem.add(str(dumped), tags=["restored_memory"])
        except Exception:
            pass


def _roundtrip_ok(obj) -> bool:
    try:
        b = dill.dumps(obj)
        _ = dill.loads(b)
        return True
    except Exception:
        return False

def _shrink_payload_for_checkpoint(payload: dict):
    """
    尝试找出导致 round-trip 失败的字段，并自动降级为“轻量可恢复checkpoint”
    """
    # 先按“最可能出问题”的对象顺序尝试剔除
    suspect_order = [
        "env",
        "players",
        "memories",
        "game_master_memory",
        "convo_externality",
        "direct_effect_externality",]
    bad_keys = []
    test_payload = dict(payload)

    # 先整体试一次
    if _roundtrip_ok(test_payload):
        return test_payload, bad_keys

    # 逐个剔除可疑字段
    for k in suspect_order:
        if k in test_payload:
            v = test_payload.pop(k)
            bad_keys.append(k)
            if _roundtrip_ok(test_payload):
                return test_payload, bad_keys

    # 如果仍失败：改成“最小checkpoint”
    minimal = {
        "step": payload.get("step"),
        "current_time": payload.get("current_time"),
        "py_random_state": payload.get("py_random_state"),
        "player_configs": payload.get("player_configs"),
        "dropped_keys": list(payload.keys()),
        "_checkpoint_type": "minimal",
    }
    # minimal 仍然不该失败，如果失败说明 dill 环境本身有问题
    if not _roundtrip_ok(minimal):
        raise RuntimeError("最小checkpoint也无法 round-trip，说明 dill/环境存在更底层问题")
    return minimal, list(payload.keys())

def save_step_checkpoint(
    *,
    checkpoint_dir: str,
    step: int,
    env,
    players,
    memories,
    clock,
    game_master_memory,
    relevant_events,
    player_status,
    direct_effect_externality,
    convo_externality,
    current_time: str,
    player_configs,
    validate_after_save: bool = True,
):
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_step_{step:06d}.pkl")
    memories_dump = {
        name: export_memory(mem)
        for name, mem in memories.items()
    }

    payload = {
        "step": step,
        "current_time": current_time,
        "clock": clock,
        "env": env,
        "players": players,
        "memories": memories,
        "game_master_memory": game_master_memory,
        "relevant_events": relevant_events,
        "player_status": player_status,
        "direct_effect_externality": direct_effect_externality,
        "convo_externality": convo_externality,
        "py_random_state": random.getstate(),
        "player_configs": player_configs,
        "memories_dumped": memories_dump,
        "game_master_memory_dumped": export_memory(game_master_memory),
        "_checkpoint_type": "full",
    }

    # ✅ 保存前先 round-trip 自检；失败就自动降级
    if not _roundtrip_ok(payload):
        safe_payload, dropped = _shrink_payload_for_checkpoint(payload)
        safe_payload["dropped_keys"] = dropped
        payload = safe_payload
        print(f"[checkpoint downgrade] step={step} dropped={dropped}")

    tmp_path = None
    try:
        with tempfile.NamedTemporaryFile(
            mode="wb",
            dir=checkpoint_dir,
            prefix=f".checkpoint_step_{step:06d}_",
            delete=False,
        ) as f:
            tmp_path = f.name
            dill.dump(payload, f)
            f.flush()
            os.fsync(f.fileno())
        os.replace(tmp_path, checkpoint_path)

        # ✅ 保存完立刻读回来自检（你想要的“马上运行”）
        if validate_after_save:
            with open(checkpoint_path, "rb") as f:
                _ = dill.load(f)
            print(f"[checkpoint self-test OK] step={step} type={payload.get('_checkpoint_type')} path={checkpoint_path}")
    finally:
        if tmp_path is not None and os.path.exists(tmp_path):
            os.remove(tmp_path)
    return checkpoint_path

def load_step_checkpoint(checkpoint_path: str):
    try:
        with open(checkpoint_path, 'rb') as f:
            payload = dill.load(f)
    except (EOFError, dill.UnpicklingError, OSError) as exc:
        raise RuntimeError(f'读取 checkpoint 失败: {checkpoint_path}\n原因：{exc}') from exc
    return payload

def _find_latest_checkpoint(checkpoint_dir: str):
    """
    在目录里找 checkpoint_step_000123.pkl 这种文件，并返回 step 最大的那个路径
    """
    if not checkpoint_dir or (not os.path.isdir(checkpoint_dir)): return None

    best_step = None
    best_path = None
    pat = re.compile(r"checkpoint_step_(\d{6})\.pkl$")

    for name in os.listdir(checkpoint_dir):
        m = pat.search(name)
        if not m:  continue
        step = int(m.group(1))
        path = os.path.join(checkpoint_dir, name)
        if best_step is None or step > best_step:
            best_step = step
            best_path = path
    return best_path

def run_simulation(
    resume_checkpoint_path: Optional[str] = None,   # 直接给某个 .pkl 的路径
    resume_checkpoint_dir: Optional[str] = None,    # 给一个目录，自动选最新的
):
  """
    支持从 checkpoint 恢复继续跑：
    - resume_checkpoint_path：精确指定某个 checkpoint 文件
    - resume_checkpoint_dir：指定 checkpoint 文件夹，自动读取最新 step 的文件
  """
  # Run the simulation for a fixed number of steps
  if resume_checkpoint_path is None and resume_checkpoint_dir is not None:
        resume_checkpoint_path = _find_latest_checkpoint(resume_checkpoint_dir)

  start_step = 0
  if resume_checkpoint_path is not None:
        print(f'检测到续跑请求，读取 checkpoint: {resume_checkpoint_path}')
        payload = load_step_checkpoint(resume_checkpoint_path)
        if payload.get("_checkpoint_type") != "full":
            print ("[resume] minimal checkpoint detected, rebuild env/players from configs...")
            # 0) 先恢复随机数状态，保证后续构建一致
            if "py_random_state" in payload:
                random.setstate(payload["py_random_state"])
            try:
                import numpy as np
                if "np_random_state" in payload:
                    np.random.set_state(payload["np_random_state"])
            except Exception:
                pass

            # 1) 取出 configs
            player_configs = payload["player_configs"]

            # 2) 重建 clock + blank_memory_factory（与新开局一致）
            clock = game_clock.MultiIntervalClock(
                start=START_TIME,
                step_sizes=[datetime.timedelta(minutes=10)]
            )
            blank_memory_factory = blank_memories.MemoryFactory(
                model=model,
                embedder=embedder,
                importance=importance_model.importance,
                clock_now=clock.now,
            )

            # 3) 用 player_configs 重建 players / memories / main_character / supported_characters
            players = []
            memories = {}
            main_character = []
            supported_characters = []

            def _infer_agent_category(c):
                """
                统一推断 agent_category：
                1) 优先读 extras 里已有字段
                2) 如果没有，就用 row_data 是否存在来判断（你工程里 NT 通常来自表格 row_data）
                """
                cat = (
                    c.extras.get("agent_category")
                    or c.extras.get("agent_type")
                    or c.extras.get("group"))
                if cat is None:
                    cat = "NT" if c.extras.get("row_data", None) is not None else "AS"
                return cat

            as_main = sum(1 for c in player_configs if c.extras.get("main_character", False) and _infer_agent_category(c) == "AS")
            nt_main = sum(1 for c in player_configs if c.extras.get("main_character", False) and _infer_agent_category(c) == "NT")
            support = sum(1 for c in player_configs if not c.extras.get("main_character", False))
            print(f"开始构建智能体实例，共有 {as_main} 个AS主角，{nt_main} 个NT主角，{support} 个配角")

            for config in player_configs:
                # 3.1 记忆
                mem = build_memory(config, blank_memory_factory)
                mem_dump = payload.get("memory_dump",{}).get(config.name,None)
                import_memory(mem, mem_dump)

                # 3.2 智能体实例
                is_main = config.extras.get("main_character", False)
                agent_category = (
                    config.extras.get("agent_category")
                    or config.extras.get("agent_type")
                    or config.extras.get("group")
                )
                if agent_category is None:
                    agent_category = "NT" if config.extras.get("row_data", None) is not None else "AS"

                if is_main:
                    if agent_category == "NT":
                        agent = _get_NT_agent(config=config, mem=mem, clock=clock)
                    else:
                        agent = _get_AS_agent(config=config, mem=mem, clock=clock)
                    main_character.append(config)
                else:
                    agent = build_support_agent(
                        config=config,
                        model=model,
                        memory=mem,
                        clock=clock,
                        update_time_interval=None
                    )
                    supported_characters.append(config)

                players.append(agent)
                memories[agent.name] = mem
                # ✅ 恢复该 agent 的记忆（用 agent.name 对齐）
                mem_dump = payload.get("memories_dump", {}).get(agent.name, None)
                import_memory(mem, mem_dump)

                # 3.3 写入 extras 里的 specific_memories（你原来 build_players_list 末尾也做了这一步）
                for extra_memory in config.extras.get("specific_memories", []):
                    mem.add(f"{extra_memory}", tags=["initial_specific_memory"])

            player_names = [p.name for p in players]
            print("build players list 完成（resume minimal）")

            # 4) 重建 game master / env（复用你现有的 build_game_master）
            print("开始构建game master（resume minimal）...")
            env, game_master_memory, relevant_events, player_status, direct_effect_externality, convo_externality = build_game_master(
                main_character=main_character,
                players=players,
                player_names=player_names,
                memories=memories,
                clock=clock,
                player_configs=player_configs,
                blank_memory_factory=blank_memory_factory,
            )

            import_memory(game_master_memory, payload.get("game_master_memory_dump"))

            # 5) 把时间拨回“续跑应该开始的时间点”
            # 注意：build_game_master 里会 clock.set(START_TIME)，所以这里必须再 set 一次
            start_step = int(payload["step"]) + 1
            resume_time = START_TIME + datetime.timedelta(minutes=10 * start_step)
            clock.set(resume_time)

            # 6) 恢复current_time
            current_time = payload.get("current_time", clock.now().strftime("%Y-%m-%d_%H-%M-%S"))

        else:
        # 从 payload 恢复所有关键对象
          env = payload["env"]
          players = payload["players"]
          memories = payload["memories"]
          clock = payload["clock"]
          game_master_memory = payload["game_master_memory"]
          relevant_events = payload["relevant_events"]
          player_status = payload["player_status"]
          direct_effect_externality = payload["direct_effect_externality"]
          convo_externality = payload["convo_externality"]
          current_time = payload.get("current_time", clock.now().strftime('%Y-%m-%d_%H-%M-%S'))
          player_configs = payload["player_configs"]

          # 恢复随机数状态（可选但建议）
          if "py_random_state" in payload:
              random.setstate(payload["py_random_state"])

        start_step = int(payload["step"]) + 1
        print(f'已从 step={payload["step"]} 恢复，将从 step={start_step} 继续运行')

        # 续跑时，checkpoint_dir沿用payload 所在目录（更直观）
        # check_point_dir = os.path.dirname(resume_checkpoint_path)
        # 续跑使用新的目录保存
        run_timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        check_point_dir = os.path.join(stored_target_folder, 'checkpoints', run_timestamp)

  else:
    clock = game_clock.MultiIntervalClock(
    start=START_TIME,
    step_sizes=[datetime.timedelta(minutes=10)]
    )

    blank_memory_factory = blank_memories.MemoryFactory(
      model=model,
      embedder=embedder,
      importance=importance_model.importance,
      clock_now=clock.now,
      )
    run_timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    current_time = clock.now().strftime('%Y-%m-%d_%H-%M-%S')
    check_point_dir = os.path.join(stored_target_folder, 'checkpoints', run_timestamp)
    players, memories, player_names, main_character, supported_characters, player_configs = build_players_list(blank_memory_factory,clock,)
    print('build players list 完成')

    print('开始构建game master...')
    t_start = time.time()
    env, game_master_memory, relevant_events, player_status, direct_effect_externality, convo_externality = build_game_master(main_character,
                                                                           players,
                                                                           player_names,
                                                                           memories,
                                                                           clock,
                                                                           player_configs,
                                                                           blank_memory_factory,)
    print(f'构建game master完成，用时: {time.time() - t_start:.2f}秒')

  print(f'开始运行模拟，episode_length={episode_length}...')
  if start_step >= episode_length:
      print(f'续跑起点 step={start_step} 已超过或等于 episode_length={episode_length}，无需继续运行')
      return {
        "env": env,
        "players": players,
        "memories": memories,
        "player_configs": player_configs,
        "game_master_memory": game_master_memory,
        "current_time": current_time,
        "convo_externality": convo_externality,
        "direct_effect_externality": direct_effect_externality,
        "relevant_events": relevant_events,
        "player_status": player_status,
        "step": start_step-1,
      }
  end_step = episode_length
  simulation_dict = {}
  for step in range(start_step, end_step):
      print(f'开始执行 step {step}...')
      t_step_start = time.time()
      env.step()
      simulation_dict = {
        "env": env,
        "players": players,
        "memories": memories,
        "player_configs": player_configs,
        "game_master_memory": game_master_memory,
        "current_time": current_time,
        "convo_externality": convo_externality,
        "direct_effect_externality": direct_effect_externality,
        "relevant_events": relevant_events,
        "player_status": player_status,
        "step": step,
      }
      checkpoint_path = save_step_checkpoint(
            checkpoint_dir=check_point_dir,
            step=step,
            env=env,
            players=players,
            memories=memories,
            clock=clock,
            game_master_memory=game_master_memory,
            relevant_events=relevant_events,
            player_status=player_status,
            direct_effect_externality=direct_effect_externality,
            convo_externality=convo_externality,
            current_time=current_time,
            player_configs=player_configs,
      )
      save_simulation_results(simulation_dict,'',check_point_dir, resume_checkpoint_dir=resume_checkpoint_dir, resume_checkpoint_path=resume_checkpoint_path)
      if step == episode_length - 1:
        _payload_test = load_step_checkpoint(checkpoint_path)
        print(f"[final checkpoint self-test OK] step={_payload_test['step']} path={checkpoint_path}")
      print(f"step {step} 完成，用时: {time.time() - t_step_start:.2f}秒 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")

  return simulation_dict

# 结果保存函数

In [15]:
def save_simulation_results(simulation_results: dict, envs: str,stored_path:str, resume_checkpoint_dir: str = None, resume_checkpoint_path: str = None):
  # Save the simulation results to a file
  env = simulation_results["env"]
  players = simulation_results["players"]
  memories = simulation_results["memories"]
  player_configs = simulation_results["player_configs"]
  game_master_memory = simulation_results["game_master_memory"]
  current_time = simulation_results["current_time"]
  current_step = simulation_results.get("step", episode_length)
  convo_externality = simulation_results["convo_externality"]
  direct_effect_externality = simulation_results["direct_effect_externality"]
  relevant_events = simulation_results["relevant_events"]
  player_status = simulation_results["player_status"]
  agent_personality = simulation_results.get("agent_personality", "Not specified")
  personality_file_contexts = {
      "model name": model_name,
      "agent personality": agent_personality,
  }

  all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

  detailed_story = '\n'.join(all_gm_memories)
  # print('len(detailed_story): ', len(detailed_story))
  # print(detailed_story)

  episode_summary = model.sample_text(
       f'Sequence of events:\n{detailed_story}'+
      '\nNarratively summarize the above temporally ordered ' +
      'sequence of events. Write it as a news report. Summary:\n',
      max_tokens=3500, terminators=()
  )
  print(episode_summary)

  player_logs = []
  player_log_names = []
  for player in players:
      name = player.name
      detailed_story = '\n'.join(memories[player.name].retrieve_recent(k=10000, add_time=True))
      summary = ''
      summary = model.sample_text(
        f'Sequence of events that happened to {name}:\n{detailed_story}'
        '\nWrite a short story that summarises these events.\n'
        ,
        max_tokens=3500, terminators=())
      all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
      all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
      player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
      player_logs.append(player_html)
      player_log_names.append(f'{name}')
  if convo_externality:
    history_sources = [env, direct_effect_externality, relevant_events, player_status, convo_externality]
  else:
    history_sources = [env, direct_effect_externality, relevant_events, player_status]
  histories_html = [html_lib.PythonObjectToHTMLConverter(history.get_history()).convert() for history in history_sources]
  histories_names = [history.name for history in history_sources]

  gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

  tabbed_html = html_lib.combine_html_pages(
      histories_html + [gm_mem_html] + player_logs,
      histories_names + ['GM'] + player_log_names,
      summary=episode_summary,
      title='Riverbend elections experiment',
  )

  def _find_previous_total_simulation_folder(checkpoint_path: str | None, checkpoint_dir: str | None, total_simulation_dir: str | None = None):    # 找到之前的结果文件
    candidates = [total_simulation_dir, checkpoint_path, checkpoint_dir]
    for candidate in candidates:
      if not candidate:
          continue
      candidate = os.path.abspath(candidate)
      # if os.path.exists(candidate):
      #   candidate = os.path.abspath(candidate)
      if os.path.isfile(candidate):
        candidate = os.path.dirname(candidate)
      if os.path.isdir(candidate) and os.path.basename(candidate) == 'total_simulation':
        return candidate
      if os.path.isdir(candidate):
        sibling = os.path.join(os.path.dirname(candidate), 'total_simulation')
        if os.path.isdir(sibling):
          return sibling
      while True:
        if os.path.basename(candidate) == 'total_simulation' and os.path.isdir(candidate):
          return candidate
        sibling = os.path.join(os.path.dirname(candidate), 'total_simulation')
        if os.path.isdir(sibling):
          return sibling
        parent = os.path.dirname(candidate)
        if parent == candidate:
          break
        candidate = parent
    return None
  def _find_previous_file(base_folder: str, suffix: str, name_hint: str | None = None):
    if not base_folder or not os.path.isdir(base_folder):
      return None
    if name_hint:
      candidate = os.path.join(base_folder, name_hint)
      if os.path.exists(candidate):
        return candidate
    matches = []
    for entry in os.listdir(base_folder):
      if entry.endswith(suffix):
        matches.append(os.path.join(base_folder, entry))
    if not matches:
      return None
    return max(matches, key=lambda p: os.path.getmtime(p))
  def _strip_html_shell(html_text: str)->str:
    if html_text.startswith(html_lib.HTML_HEAD):
      html_text = html_text[len(html_lib.HTML_HEAD):]
    if html_text.endswith(html_lib.HTML_TAIL):
      html_text = html_text[:-len(html_lib.HTML_TAIL)]
    return html_text
  def _extract_tab_content(html_text: str)->dict:
    body = _strip_html_shell(html_text)
    pattern = re.compile(r'<div id=\"([^\"]+)\" class=\"tabcontent\">(.*?)</div>', re.DOTALL)
    return {name: content for name, content in pattern.findall(body)}
  def _merge_html_tabs(previous_html: str, new_html: str, tab_names: list[str]):
    previous_tabs = _extract_tab_content(previous_html)
    new_tabs = _extract_tab_content(new_html)
    merged_tabs_names = list(tab_names)
    for tab_name in previous_tabs:
      if tab_name not in merged_tabs_names:
        merged_tabs_names.append(tab_name)
    merged_pages = []
    for tab_name in merged_tabs_names:
      previous_content = previous_tabs.get(tab_name)
      new_content = new_tabs.get(tab_name)
      if previous_content and new_content:
        merged_pages.append(previous_content + '<hr />' + new_content)
      elif new_content:
        merged_pages.append(new_content)
      else:
        merged_pages.append(previous_content or '')
    return html_lib.combine_html_pages(
        merged_pages,
        merged_tabs_names,
        summary=episode_summary,
        title='Riverbend elections experiment',
    )

  previous_total_simulation = _find_previous_total_simulation_folder(
      resume_checkpoint_path or globals().get('checkpoint_file'),
      resume_checkpoint_dir or globals().get('checkpoint_folder'),
      globals().get('total_simulation_folder')
  )
  if previous_total_simulation:
    previous_html_file = _find_previous_file(
        previous_total_simulation,
        suffix='.html',
        name_hint=f'{current_time}.html',
    )
    if previous_html_file:
      print(f'检测到之前的 total_simulation 结果，合并历史记录：{previous_html_file}')
      try:
        with open(previous_html_file, 'r', encoding='utf-8') as f:
          previous_html = f.read()
        tabbed_html = _merge_html_tabs(previous_html, tabbed_html, histories_names + ['GM'] + player_log_names)
      except Exception:
        print(f'合并之前的 total_simulation 结果失败，继续保存当前结果即可：{previous_html_file}')
        pass

  tabbed_html = html_lib.finalise_html(tabbed_html)

  # @title Save the output to a file
  # current_time = clock.now().strftime('%Y-%m-%d_%H-%M-%S')

  output_file = f'{current_time}.html'  # @param {type: 'string'}
  stored_target_folder_env = os.path.join(stored_path, envs)
  output_file = os.path.join(stored_target_folder_env, output_file)

  dir_path = os.path.dirname(output_file)
  os.makedirs(dir_path, exist_ok=True)

  try:
    with open(output_file, 'w', encoding='utf-8') as f:
      f.write(tabbed_html)
  except:
    try:
      with open(f'{output_file}_1.html', 'w', encoding='utf-8') as f:
        f.write(tabbed_html)
    except:
        tabbed_html = tabbed_html.encode('utf-8', 'replace').decode('utf-8')
        with open(f'{output_file}_2.html', 'w', encoding='utf-8') as f:
          f.write(tabbed_html)

  def track_each_action_delta_change(action_sequences:list[str], value_tracker: value_comp.ValueTracker):
    # 对比相邻两步行动的 delta（欲望变化值），找出“变得更小”的项目，从而识别哪些欲望因为某个 action 而下降了
    individual_delta_tracker = value_tracker.get_individual_delta_tracker()
    previous_one_delta = None
    change_in_desire = dict()
    for index, action in enumerate(action_sequences):
      if index == 0: # skip the first action
        previous_one_delta = individual_delta_tracker.get(index, None) # get the first delta
        continue
      current_one_delta = individual_delta_tracker.get(index, None) # get the current delta
      if previous_one_delta and current_one_delta: # if both are not None
        # if delta smaller than previous one, record it
        delta_change = [k for k in current_one_delta.keys() if current_one_delta[k] < previous_one_delta[k]]
        # print(f"Action: {action}, Delta Change: {delta_change}")
      change_in_desire[index] = delta_change

  def summarise_value(player: EntityWithComponents, other_players: list = None, step: int = 0):
      value_tracker = None
      try:
        value_tracker = player.get_component('ValueTracker', type_= value_comp.ValueTracker)
      except KeyError:
        value_tracker = None

      # action_seq = value_tracker.get_action_sequence()
      json_result = dict()
      json_result['save_timestamp'] = current_time
      json_result['start_time'] = str(EXP_START_TIME)
      # json_result['action_sequence'] = [
      #   {'timestamp': str(each_act_dict['timestamp']),
      #     'action': each_act_dict['action'].strip()}
      #     for each_act_dict in action_seq]
      # json_result['step'] = episode_length
      json_result['step'] = step

      json_result['value_tracker_missing'] = value_tracker is None
      if value_tracker is None:
          json_result['action_sequence'] = ['no action sequence']
          json_result['whole_delta'] = {'no whole delta': -1}
          json_result['individual_delta'] = {'no individual delta': -1}
          json_result['individual_desire'] = {'no individual desire': -1}
          json_result['individual_qualitative_desire'] = {'no individual qualitative desire': -1}
      else:
          action_seq = value_tracker.get_action_sequence()
          json_result['action_sequence'] = [
            {'timestamp': str(each_act_dict['timestamp']),
              'action': each_act_dict['action'].strip()}
              for each_act_dict in action_seq]
          whole_delta_tracker = value_tracker.get_whole_delta_tracker()#所有desire偏差总和
          # print(f"whole_delta_tracker: {whole_delta_tracker}")
          json_result['whole_delta'] = {int(k): float(v) for k, v in whole_delta_tracker.items()}

          individual_delta_tracker = value_tracker.get_individual_delta_tracker()#各个desire的偏差
          # print(f"individual_delta_tracker: {individual_delta_tracker}")
          json_result['individual_delta'] = {
              int(k_step): {delta: float(value) for delta, value in delta_value_pair.items()}
              for k_step, delta_value_pair in individual_delta_tracker.items()
          }

          individual_desire_tracker = value_tracker.get_individual_desire_tracker()
          # print(f"individual_desire_tracker: {individual_desire_tracker}")
          json_result['individual_desire'] = {
              int(k_step): {desire: int(value) for desire, value in desire_value_pair.items()}
              for k_step, desire_value_pair in individual_desire_tracker.items()
          }

          individual_qualitative_desire_tracker = value_tracker.get_individual_qualitative_desire_tracker()
          # print(f"individual_qualitative_desire_tracker: {individual_qualitative_desire_tracker}")
          json_result['individual_qualitative_desire'] = {int(k): v for k,v in individual_qualitative_desire_tracker.items()}

          expected_values = value_tracker.get_expected_value_dict()
          # print(f"expected_values: {expected_values}")
          json_result['expected_values'] = {desire_name: float(exp_value) for desire_name, exp_value in expected_values.items()}

      if player.name == 'Sheldon':
        profile = agent_desire_profile_AS['visual_desire_string']
      else:
        profile = agent_desire_profile_NT['visual_desire_string']
      json_result['profile'] = profile

      initial_value = {k: float(v) for k, v in numerical_desire.items()}
      json_result['initial_value'] = initial_value

      sampled_background = shared_context
      json_result['sampled_background'] = sampled_background
      json_result['whole_background(This simulation)'] = '\n'.join(shared_memory)

      # 尝试查找名为 'Alice' 的智能体，如果不存在则跳过
      alice_list = [i for i in player_configs if i.name == 'Alice']
      if alice_list:
          alice = alice_list[0]
          alice_dict = alice.to_dict()
          json_result['Alice_setting'] = alice_dict
      else:
          json_result['Alice_setting'] = None

      json_result['wanted_desires'] = wanted_desires
      json_result['hidden_desires'] = hidden_desires
      if player.name == 'Sheldon':
          json_result['visual_desires_dict'] = agent_desire_profile_AS['visual_desires_dict']
          json_result['hidden_desires_dict'] = agent_desire_profile_AS['hidden_desires_dict']
          json_result['selected_desire_dict'] = agent_desire_profile_AS['selected_desire_dict']
          json_result['all_desire_traits_dict'] = agent_desire_profile_AS['all_desire_traits_dict']
          json_result['visual_desire_string'] = agent_desire_profile_AS['visual_desire_string']
      else:
          json_result['visual_desires_dict'] = agent_desire_profile_NT['visual_desires_dict']
          json_result['hidden_desires_dict'] = agent_desire_profile_NT['hidden_desires_dict']
          json_result['selected_desire_dict'] = agent_desire_profile_NT['selected_desire_dict']
          json_result['all_desire_traits_dict'] = agent_desire_profile_NT['all_desire_traits_dict']
          json_result['visual_desire_string'] = agent_desire_profile_NT['visual_desire_string']

      json_result['model_name'] = model_name

      if other_players:
          other_players_dict = {player.name: player.to_dict() for player in other_players}
          json_result['other_players'] = other_players_dict

      if Use_Previous_profile:
        json_result['previous_profile_file'] = previous_profile_file
      return json_result
  def merge_step_list(existing_list, new_list):
    # 合并两个 step 列表，优先保留已有历史，追加新的 action，避免覆盖早期步骤
    if not isinstance(existing_list, list) or not isinstance(new_list, list):
        return new_list
    def _is_placeholder(item):
      return isinstance(item, str) and item.strip().lower() == 'no action sequence'
    existing_list = [item for item in existing_list if not _is_placeholder(item)]
    new_list = [item for item in new_list if not _is_placeholder(item)]
    if not existing_list and not new_list:
      return ['no action sequence']
    merged = list(existing_list)
    def _action_key(item):
      if isinstance(item, dict):
        return (item.get('timestamp'), item.get('action'))
      return item
    existing_keys = {_action_key(item) for item in existing_list}
    for item in new_list:
      if _action_key(item) not in existing_keys:
        merged.append(item)
    return merged

  def merge_step_dict(existing_dict, new_dict):
      # 合并两个 step 字典，保留更完整的那个，且在更完整的后续部分追加到现有字典后面
      if not isinstance(existing_dict, dict) or not isinstance(new_dict, dict):
          return new_dict
      merged_dict = dict(existing_dict)
      merged_dict.update(new_dict)
      return merged_dict

  def _merge_simulation_json(existing_data: dict, new_data: dict):
      # 合并两个 simulation json 数据
      if not isinstance(existing_data, dict) or not existing_data:
          return new_data
      merged_data = dict(existing_data)
      merged_data['save_timestamp'] = new_data.get('save_timestamp', merged_data.get('save_timestamp'))
      merged_data['step'] = max(existing_data.get('step', -1), new_data.get('step', -1))
      merged_data['action_sequence'] = merge_step_list(
          existing_data.get('action_sequence', []),
          new_data.get('action_sequence', []),
      )
      for key in (
          "whole_delta",
          "individual_delta",
          "individual_desire",
          "individual_qualitative_desire",
      ):
          merged_data[key] = merge_step_dict(
              existing_data.get(key),
              new_data.get(key),
          )
      for key,value in new_data.items():
          if key not in merged_data:
              merged_data[key] = value
      if 'expected_values' in new_data:
          merged_data['expected_values'] = new_data['expected_values']
      return merged_data


  personality_path = f'{current_time}.json'
  personality_file = os.path.join(stored_target_folder_env, personality_path)
  try:
      with open(personality_file, 'w') as f:
          json.dump(personality_file_contexts, f, indent=4)
  except:
      with open(personality_file, 'w', encoding='utf-8') as f:
          json.dump(personality_file_contexts, f, indent=4)

  result_files = []
  for each_agent in players:
    value_result = f'{current_time}_{each_agent.name}.json'
    value_result_file = os.path.join(stored_target_folder_env, value_result)
    new_payload = summarise_value(each_agent, step=current_step)
    merged_payload = new_payload
    # previous_value_result_file = None
    # if previous_total_simulation:
    #   previous_value_result_file = _find_previous_file(
    #       previous_total_simulation,
    #       suffix=f'_{each_agent.name}.json',
    #   )
    # existing_source = None
    if os.path.exists(value_result_file):
    #     existing_source = value_result_file
    # elif previous_value_result_file:
    #     existing_source = previous_value_result_file
    # if existing_source:
        try:
            with open(value_result_file, 'r', encoding='utf-8') as f:
            # with open(existing_source, 'r', encoding='utf-8') as f:
                existing_payload = json.load(f)
            merged_payload = _merge_simulation_json(existing_payload, new_payload)
        except Exception:
            merged_payload = new_payload
    try:
      with open(value_result_file, 'w') as f:
        json.dump(merged_payload, f, indent=4, ensure_ascii=False)
    except:
      with open('filename.json', 'w', encoding='utf-8') as f:
        json.dump(merged_payload, f, indent=4, ensure_ascii=False)
    result_files.append(value_result_file)
  return result_files

# 运行实验单元并保存结果

In [16]:
save_files = {}
result = run_simulation(
    resume_checkpoint_dir=checkpoint_folder,
    resume_checkpoint_path=checkpoint_file,
)
save_path = save_simulation_results(result, stored_path=stored_target_folder, envs='total_simulation',)

print(f" Finished simulation")

构建自闭症智能体成功
构建典型发展智能体成功
构建教师智能体
开始构建智能体实例，共有 1 个AS主角，15 个NT主角，1 个配角
  正在构建AS智能体: Sheldon...
      [build_memory] 开始为 Sheldon 构建记忆 (main_character=True)...
      [build_memory] 使用 FormativeMemoryFactoryWithoutBackground (不会调用LLM生成backstory)
      [build_memory] 调用 make_memories...


D:\Code\Autism-simulation\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


      [build_memory] make_memories 完成，用时: 0.46秒
      [build_memory] 总用时: 0.46秒
    构建记忆完成，用时: 0.46秒
      正在调用 build_D2A_agent 构建 Sheldon...
      背景知识长度: 95 字符
        [build_D2A_agent] 开始预处理value信息...
        [build_D2A_agent] 预处理完成，用时: 0.00秒
        [build_D2A_agent] 开始创建desire组件 (共 5 个)...
        [build_D2A_agent] desire组件创建完成，用时: 0.00秒
      build_D2A_agent 完成，用时: 0.00秒
    构建智能体完成，用时: 0.00秒
  正在构建NT智能体: Nate...
      [build_memory] 开始为 Nate 构建记忆 (main_character=True)...
      [build_memory] 使用 FormativeMemoryFactoryWithoutBackground (不会调用LLM生成backstory)
      [build_memory] 调用 make_memories...
      [build_memory] make_memories 完成，用时: 0.13秒
      [build_memory] 总用时: 0.13秒
    构建记忆完成，用时: 0.13秒
      正在为 Nate 生成traits背景知识...
      traits背景知识生成完成，用时: 0.00秒，长度: 6833 字符
      正在调用 build_D2A_agent 构建 Nate...
        [build_D2A_agent] 开始预处理value信息...
        [build_D2A_agent] 预处理完成，用时: 0.00秒
        [build_D2A_agent] 开始创建desire组件 (共 5 个)...
        [build_D2A_agent] desire组件创建完成，用时: 0

D:\Code\Autism-simulation\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


  正在构建NT智能体: Chloe...
      [build_memory] 开始为 Chloe 构建记忆 (main_character=True)...
      [build_memory] 使用 FormativeMemoryFactoryWithoutBackground (不会调用LLM生成backstory)
      [build_memory] 调用 make_memories...
      [build_memory] make_memories 完成，用时: 0.17秒
      [build_memory] 总用时: 0.17秒
    构建记忆完成，用时: 0.17秒
      正在为 Chloe 生成traits背景知识...
      traits背景知识生成完成，用时: 0.00秒，长度: 6791 字符
      正在调用 build_D2A_agent 构建 Chloe...
        [build_D2A_agent] 开始预处理value信息...
        [build_D2A_agent] 预处理完成，用时: 0.00秒
        [build_D2A_agent] 开始创建desire组件 (共 5 个)...
        [build_D2A_agent] desire组件创建完成，用时: 0.00秒
      build_D2A_agent 完成，用时: 0.00秒
    构建智能体完成，用时: 0.00秒


...

build players list 完成
开始构建game master...
game master初步记忆构建用时： 0.0


D:\Code\Autism-simulation\concordia\associative_memory\associative_memory.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self._memory_bank = pd.concat(


game master记忆添加每个智能体的记忆构建用时： 1.3784799575805664
构建game master完成，用时: 36.56秒
开始运行模拟，episode_length=60...
开始执行 step 0...


ERROR:absl:Error in task <bound method PlayerStatus.name of <concordia.components.game_master.player_status.PlayerStatus object at 0x000001EB3B272B40>>.update
Traceback (most recent call last):
  File "D:\Code\Autism-simulation\concordia\utils\concurrency.py", line 60, in _run_task
    return fn()
           ^^^^
  File "D:\Code\Autism-simulation\concordia\utils\helper_functions.py", line 144, in apply_recursively
    getattr(parent_component, function_name)()
  File "D:\Code\Autism-simulation\concordia\components\game_master\player_status.py", line 103, in update
    prompt.open_question(
  File "D:\Code\Autism-simulation\concordia\document\interactive_document.py", line 176, in open_question
    response = self._model.sample_text(
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\Code\Autism-simulation\concordia\language_model\base_gpt_model.py", line 101, in sample_text
    raise ValueError("Empty response from LLM")
ValueError: Empty response from LLM


ValueError: Empty response from LLM

In [ ]:
print("结果保存路径：", save_path)